In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt


**Pre Processing:**

In [3]:
## Droping cabin since 77% is NaN, and Ticket and Name are not informative.
## Turning Sex and Embarked into One Hot encodings

from sklearn.preprocessing import OneHotEncoder

def pre_proccess(up_set, isTrain=True):
  avg_age = np.floor(up_set['Age'][up_set['Age'].isna()==False].mean())
  up_set['Age'][up_set['Age'].isna()] = avg_age

  up_set_proc = up_set.drop(columns=['Ticket', 'Cabin', 'Name','Embarked'])
  up_set_proc['Fare'][up_set_proc['Fare'].isna()] = 0
  # if isTrain:
  #   drop_idx = up_set_proc[up_set_proc['Embarked'].isna()].index
  #   up_set_proc = up_set_proc.drop(index=drop_idx)
  up_set_proc['Sex'][up_set_proc['Sex']=='male'] = 0
  up_set_proc['Sex'][up_set_proc['Sex']=='female'] = 1
  up_set_proc['Sex'] = up_set_proc['Sex'].astype(int)

  # enc_features = OneHotEncoder().fit_transform(up_set_proc[['Embarked']]).toarray()
  # enc_features = np.array(list(zip(enc_features)))
  # up_set_proc = up_set_proc.drop(columns='Embarked')
  # up_set_proc['Embarked'] = [x[0][0] for x in list(zip(enc_features))]
  return up_set_proc

In [8]:
## Droping cabin since 77% is NaN, and Ticket and Name are not informative.

def pre_proccess_2(up_set, isTrain=True):
  # Setting all NaN ages to the average age. The same with Fare.
  avg_age = np.floor(up_set['Age'][up_set['Age'].isna()==False].mean())
  up_set['Age'][up_set['Age'].isna()] = avg_age
  avg_fare = np.floor(up_set['Fare'][up_set['Fare'].isna()==False].mean())
  up_set['Fare'][up_set['Fare'].isna()] = avg_fare

  # Droping irrelevant columns, and changing catagorical columns to numeric vals.
  up_set_proc = up_set.drop(columns=['Ticket','Embarked', 'Name', 'Cabin'])
  up_set_proc['Sex'][up_set_proc['Sex']=='male'] = 0
  up_set_proc['Sex'][up_set_proc['Sex']=='female'] = 1
  up_set_proc['Sex'] = up_set_proc['Sex'].astype(int)


  # Max Abs normalization:
  for col in ['Age', 'Fare', 'SibSp', 'Parch']:
    up_set_proc[col] = up_set_proc[col] /up_set_proc[col].abs().max()


  return up_set_proc

**Prepering Train/Val/Test sets:**

In [82]:
train_set_up = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Kaggle/Titanic/train.csv', index_col='PassengerId')
test_set_up = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Kaggle/Titanic/test.csv', index_col='PassengerId')
train_set_p = pre_proccess_2(train_set_up)
test_set_p = pre_proccess_2(test_set_up, isTrain=False)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarn

**Model Training:**

**Gradient Boosting Tree:**

In [79]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
train_set, val_set = train_test_split(train_set_p,test_size=300, random_state=42)

train_set_y = train_set['Survived']
train_set_x = train_set.drop(columns='Survived')
val_set_y = val_set['Survived']
val_set_x = val_set.drop(columns='Survived')
loss_list = []
for est in range(50,800,50):
  for lr in [0.1,0.01,0.001,0.0001]:
    for md in [3,4,5,7,10.15,20,25,30]:
      model_gb = GradientBoostingClassifier(max_depth=md,n_estimators=est, learning_rate=lr)
      model_gb.fit(train_set_x, train_set_y)
      loss_list.append((est,lr,md,model_gb.score(val_set_x, val_set_y)))

In [80]:
loss_list.sort(key= lambda x: x[3], reverse=True)
loss_list[:10]

[(550, 0.01, 5, 0.8433333333333334),
 (600, 0.01, 5, 0.8433333333333334),
 (50, 0.1, 5, 0.84),
 (250, 0.01, 3, 0.84),
 (300, 0.01, 3, 0.84),
 (350, 0.01, 3, 0.84),
 (200, 0.01, 3, 0.8366666666666667),
 (650, 0.01, 5, 0.8366666666666667),
 (700, 0.01, 5, 0.8366666666666667),
 (200, 0.1, 5, 0.8333333333333334)]

In [83]:
train_set_y = train_set['Survived']
train_set_x = train_set.drop(columns='Survived')
val_set_y = val_set['Survived']
val_set_x = val_set.drop(columns='Survived')
md = 5
est = 600
lr = 0.01

model = GradientBoostingClassifier(max_depth=md,n_estimators=est, learning_rate=lr)
model.fit(train_set_x, train_set_y)
y_pred = model.predict(test_set_p)
test_set_p['Survived'] = y_pred
final_df = test_set_p['Survived']
final_df.to_csv(f'prediction_{md}_{est}_{lr}.csv')

**XGBooster:**

In [45]:
loss_list = []
train_set_y = train_set['Survived']
train_set_x = train_set.drop(columns='Survived')
val_set_y = val_set['Survived']
val_set_x = val_set.drop(columns='Survived')
for est in range(50,800,50):
  for lr in [0.1,0.01,0.001]:
    for md in [3,4,5,7,10,15,20,25,30]:
      model_xgd =xgb.XGBClassifier(max_depth=md,n_estimators=est, learning_rate=lr)
      model_xgd.fit(train_set_x, train_set_y)
      loss_list.append((est,lr, md,model_xgd.score(val_set_x, val_set_y)))

In [ ]:
loss_list.sort(key= lambda x: x[3], reverse=True)
loss_list[:10]

**Random Forrest:**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

train_set, val_set = train_test_split(train_set_p,test_size=300,random_state=10)
train_set_y = train_set['Survived']
train_set_x = train_set.drop(columns='Survived')
val_set_y = val_set['Survived']
val_set_x = val_set.drop(columns='Survived')

loss_list = []
for est in range(50,800,50):
    for md in [3,4,5,7,10,15,20,25,30]:
      for bs in [True, False]:
        model_rf =RandomForestClassifier(max_depth=md, n_estimators=est, bootstrap=bs,random_state=10)
        model_rf.fit(train_set_x, train_set_y)
        loss_list.append((est, md, bs,model_rf.score(val_set_x, val_set_y)))

In [ ]:
loss_list.sort(key= lambda x: x[3], reverse=True)
loss_list[:10]

**Predict:**

In [62]:
train_set_y = train_set['Survived']
train_set_x = train_set.drop(columns='Survived')
val_set_y = val_set['Survived']
val_set_x = val_set.drop(columns='Survived')

model = RandomForestClassifier(max_depth=5, n_estimators=50, bootstrap=False,random_state=10)
model.fit(train_set_x, train_set_y)
y_pred = model.predict(test_set_p)
test_set_p['Survived'] = y_pred
final_df = test_set_p['Survived']
final_df.to_csv('prediction.csv')

0.8433333333333334